# MNE-Python: From raw data to epochs and evoked responses (ERF/ERP)

`
Authors:
Alexandre Gramfort
Denis A. Engemann
Jona Sassenhagen
Richard Höchenberger
`

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

First, load the mne package:

In [ ]:
import mne

We set the logging level to 'warning' so the output will be less verbose and we can focus on the important bits 🧐

In [ ]:
mne.set_log_level('warning')

### Remember: If you need help, just ask... the machine!

In [ ]:
mne.pick_types?

## Access raw data

Now we import the `sample` dataset.

It will be downloaded automatically (approx. 2 GB)

In [ ]:
from mne.datasets import sample
data_path = sample.data_path()

raw_fname = os.path.join(data_path, 'MEG/sample/sample_audvis_filt-0-40_raw.fif')

In [ ]:
print(raw_fname)

Read data from file:

In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=True)
print(raw)

Note that by default, the data will actually not be loaded into memory automatically to preserve memory. To actually load the data, we have to pass `preload=True`.

Now let's look at the measurement info. It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.


In [ ]:
print(raw.info)

raw.info is just a dictionary:

In [ ]:
isinstance(raw.info, dict)

So we can access its elements this way:

In [ ]:
raw.info['sfreq']  # Sampling frequency

In [ ]:
raw.info['bads']  # list of marked bad channels

Next let's see what channels are present in the data. We simply have to take a loot at the `raw.ch_names` attribute.

In [ ]:
raw.ch_names

You can index it as a list

In [ ]:
raw.ch_names[42]

In [ ]:
raw.ch_names[:10]

Channel type of a specific channel

In [ ]:
channel_type = mne.io.pick.channel_type(info=raw.info, idx=75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.io.pick.channel_type(info=raw.info, idx=320)
print('Channel #320 is of type:', channel_type)

`raw.info['chs']` contains all the details about the sensors (type, locations, coordinate frame etc.)

In [ ]:
len(raw.info['chs'])

In [ ]:
type(raw.info['chs'])

In [ ]:
raw.info['chs'][0]

In [ ]:
raw.info['chs'][330]

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='grad');

## Accessing the data

To access the data, just use the `[]` syntax as to access any element of a list, dict etc.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

In [ ]:
times

Note that it returns both the data and the times array.

# Visualizing raw data

Note : we will use the QT backend from matplotlib that will open a separate window.

In [ ]:
%matplotlib qt
fig = raw.plot()

In [ ]:
fig = raw.copy().pick_types(meg=False, eeg=True).plot()

## Filtering

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=True)

In [ ]:
raw_beta = raw.copy().filter(l_freq=13, h_freq=30, verbose=True)

In [ ]:
print(raw_beta.info)  # note the update of raw.info['lowpass'] and raw.info['highpass']

In [ ]:
raw_beta.plot()

In [ ]:
raw_beta.filter?

### Exercise
Plot the 10 first seconds of the stimutation channel `STI 014` just using matplotlib.

Tips:

- Find the channel index using `raw.ch_names.index('STI 014')`
- Get the data for this channel
- Plot it using `plt.plot`

## Define and read epochs

### First, extract events.

In [ ]:
events = mne.find_events(raw, stim_channel='STI 014', verbose=True)

In [ ]:
events.shape

In [ ]:
type(events)

In [ ]:
print(events[:5])  # events is a 2d array, (time, previous, trigger)

In [ ]:
len(events[events[:, 2] == 4])

In [ ]:
len(events)

Let's visualize the paradigm:

In [ ]:
%matplotlib inline
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'])

For describing which event IDs code for which experimental events or conditions, we use a Python dictionary to store the mapping. The dictionary keys can contain `/` for grouping of sub-conditions.

In [ ]:
event_id = {"visual/left": 3, "visual/right": 4,
            "auditory/left": 1, "auditory/right": 2}

fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], event_id=event_id)

The events can be visualized together with the raw data:

In [ ]:
fig = raw.plot(event_id=event_id, events=events)

### Create epochs

First, define parameters: start, stop, and baseline period of the epochs.

In [ ]:
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5   # end of each epoch (500ms after the trigger)
baseline = (None, 0)  # from the first time instant to the trigger pulse

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers, and EOG:

In [ ]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

In [ ]:
# we are picky again, this time we select MEG and EOG channels
picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                           stim=False, exclude='bads')

Extract epochs:

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks_meg, baseline=baseline,
                    reject=reject)

In [ ]:
print(epochs)

Remove bad epochs based on the `reject` parameter we passed to `Epochs`.

In [ ]:
epochs.drop_bad()

Explore the epochs namespace

Hit ``epochs.<TAB>``

See how epochs were dropped

In [ ]:
fig = epochs.plot_drop_log()

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/plot_visualize_epochs.html) for options on how to visualize epochs.

Here is just an illustration to make a so-called ERP/ERF image:

In [ ]:
fig = raw.plot_psd(fmax=40)

In [ ]:
figs = epochs.plot_image()

In [ ]:
%matplotlib qt
fig = epochs.plot()

### Average the epochs to get the evoked response (ERF/ERP), and plot it!

In [ ]:
evoked = epochs.average()
print(evoked)

In [ ]:
%matplotlib inline
fig = evoked.plot(spatial_colors=True)

This created an average across **all** conditions. Let's now estimate evoked responses for **individual** conditions.

In [ ]:
print(event_id)

In [ ]:
fig = epochs['auditory/left'].average().plot(spatial_colors=True)

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

Remember `/` serves as a grouping operator. To calculate the evoked response across **all** "left" stimulations, do the following:

In [ ]:
fig = epochs['left'].average().plot(spatial_colors=True);  # note the legend

In [ ]:
# remember ...
event_id

In [ ]:
epochs[0]  # first epoch

In [ ]:
epochs[:10]  # first 10 epochs

In [ ]:
epochs['visual/left']  # epochs for the left visual condition

In event_id, `/` selects conditions in a hierarchical way, e.g. here, "auditory" vs. "visual", "left" vs. "right", and MNE can select them individually.

In [ ]:
evoked_auditory_left = epochs['auditory/left'].average().pick_types(meg='grad')
evoked_auditory_left.crop(None, 0.2) # Beginning of evoked until 0.2s after stimulus onset.
fig = evoked_auditory_left.plot(spatial_colors=True)

In [ ]:
epochs['visual']  # epochs for the visual condition (either left or right)

In [ ]:
epochs['left']

To access the data of some epochs use the `get_data` method.


In [ ]:
epochs_data = epochs.get_data()
type(epochs_data), epochs_data.shape

`epochs_data` is a 3D array of dimension (239 epochs, 306 channels, 106 time instants).


## Visualize Topographies

First, let's plot an illustration of the sensor locations on the scalp

In [ ]:
fig = raw.plot_sensors(ch_type='mag')
fig = raw.plot_sensors(ch_type='grad')

Now, let's plot the scalp topography at different time points. These topography plots called `topomap` in the MNE jargon.


In [ ]:
fig = evoked.plot_topomap(ch_type='mag', times=[0.05, 0.1, 0.15])

In [ ]:
fig = evoked.plot_topomap(ch_type='grad', times=[0.05, 0.1, 0.15])

In [ ]:
import numpy as np


times = np.linspace(0.05, 0.15, 10)

for ch_type in ('mag', 'grad'):
    fig = evoked.plot_topomap(times=times, ch_type=ch_type)

Topoplot and time series can also be shown in one single "joint" plot. By default, MNE will plot the topographies at the peaks of the time series signal.

In [ ]:
figs = evoked.plot_joint()

But of course, you can also specify custom time points for the topomaps.

In [ ]:
figs = evoked.plot_joint(times=[0.1, 0.3])

Let's visualize topomaps for all experimental conditions.

In [ ]:
for condition in event_id:
    fig = epochs[condition].average().plot_topomap(times=[0.1, 0.15], title=condition)

### Compute a contrast:

In [ ]:
evoked1 = epochs['left'].average()
evoked2 = epochs['right'].average()

contrast = mne.combine_evoked([evoked1, evoked2], weights=[1, -1])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [ ]:
print(evoked1.nave)  # average of 55 epochs
print(contrast.nave)  # average of 116 epochs

In [ ]:
print(contrast)

In [ ]:
fig = contrast.plot_joint()

### EXERCISE
- Extract Epochs restricted to magnetometers on unfiltered data (`sample_audvis_raw.fif`)
- Construct epochs with a whole-epoch baseline. Then, high-pass filter raw data with a 1 Hz cutoff, construct epochs from that. Compare the resulting Evokeds (filter vs. baseline)
- Plot the difference between all *visual* and all *auditory* stimulus presentations
- Recompute everything for EEG